In [1]:
!pip install strands-agents[mistral] python-dotenv

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.0 MB/s  0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: pydanticm━━━━━━━━━━━━━━━━━━━━━━━━━━━━  6/21 [httpcore]
    Found existing installation: pydantic 2.9.2━━━━━━━━━━━━━━━  6/21 [httpcore]
    Uninstalling pydantic-2.9.2:0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━  6/21 [httpcore]
      Successfully uninstalled pydantic-2.9.2━━━━━━━━━━━━━━━━━  6/21 [httpcore]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21/21 [strands-agents]m [strands-agents]strumentation]
ERROR: pip's dependency resolver does not currently take into account all the packages that ar

In [1]:
import json
import os
import sys
import dotenv

import pprint

import yaml
from pathlib import Path
from typing import Dict, List, Tuple, TypeVar
from tqdm import tqdm

# Pydantic for structured data
from pydantic import BaseModel, Field

# Strands for AI agents
from strands.agent import Agent
from strands.models.mistral import MistralModel

from collections import Counter

sys.path.append('..')

from src.utils import (
    save_json,
    read_json,
    load_file_content,
    get_job_paths,
    get_training_paths,
    sanity_check,
	chat_with_persona,
    track_api_call,  # Cost tracking from utils
    print_cost_summary,  # Cost summary from utils
    reset_cost_tracker  # Reset cost tracker from utils
)

from src.my_utils import (
    display_markdown_file,
    call_mistral,
    get_agent,
    batch_extract,
    compute_stat_for_multi_items
)

from src.models.job_info import JobInfo
from src.prompts.job_extraction_prompt import JOB_EXTRACTION_PROMPT

# Load API key from .env file
dotenv.load_dotenv("../env")

# Check if we're good to go
if not os.getenv("MISTRAL_API_KEY"):
    print("❌ No MISTRAL_API_KEY found!")
    print("Create an env file with your API key")
else:
    print("✅ API key found, we're ready to roll")

✅ API key found, we're ready to roll


In [2]:
with open("../src/config.yaml", "r") as f:
    config = yaml.safe_load(f)

job_data_version = config["job_data_version"]
print(f"job_data_version version: {job_data_version}")

job_data_version version: v4


In [3]:
DATA_JOBS_DIR = Path('../data_jobs')

In [4]:
def extract_job_info(
    path: Path,
    model: str = "mistral-small-latest"
) -> JobInfo:
    """Extract job info from file using Job Extraction Agent"""
    text = load_file_content(path)
    prompt = JOB_EXTRACTION_PROMPT + text

    extraction_agent = get_agent(model_id=model, temperature=0.0)
    result = extraction_agent.structured_output(output_model=JobInfo, prompt=prompt)
    result.domains = []
    result.required_skills = {}
    result.required_skills_domains = []

    if hasattr(extraction_agent, 'last_response'):
        track_api_call(extraction_agent.last_response, model)

    return result

In [5]:
# Process all jobs
print("📂 Processing Jobs...")

filename = f"extracted_jobs_{job_data_version}.json"
jobs_save_path = DATA_JOBS_DIR / filename

jobs_data = batch_extract(
    get_job_paths(),
    extract_job_info,
    jobs_save_path,
    show_cost_every=50  # Show cost update every 50 items
)

# Convert to JobInfo objects
jobs_info = {
    job_id: JobInfo.model_validate_json(data)
    for job_id, data in jobs_data.items()
}

print(f"✅ Extracted {len(jobs_info)} jobs")
print("\n" + "="*50)

📂 Processing Jobs...
Processing 200 files (0 already cached)


 25%|██▌       | 50/200 [01:09<03:00,  1.20s/it]


💰 Cost update after 50 new items:
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000



 50%|█████     | 100/200 [02:27<02:25,  1.45s/it]


💰 Cost update after 100 new items:
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000



 65%|██████▌   | 130/200 [03:14<01:56,  1.66s/it]/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `list[str]` - serialized value may not be as expected [field_name='required_skills_domains', input_value={}, input_type=dict])
  return self.__pydantic_serializer__.to_python(
 75%|███████▌  | 150/200 [03:44<01:29,  1.80s/it]


💰 Cost update after 150 new items:
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000



100%|██████████| 200/200 [04:58<00:00,  1.49s/it]


💰 Cost update after 200 new items:
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000


✅ Processed 200 new items
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000
✅ Extracted 200 jobs



In [6]:
filename = f"extracted_jobs_{job_data_version}.json"
jobs_save_path = DATA_JOBS_DIR / filename
jobs_data = read_json(jobs_save_path)

In [7]:
def compute_job_data_stastistics(jobs_data):
    # Extract infos from the dictionary
    work_types = []
    education_level_requireds = []
    years_of_experience_requireds = []
    for job_data in jobs_data.values():
        data = json.loads(job_data)
        work_types.append(data['work_type'])
        education_level_requireds.append(data['education_level_required'])
        years_of_experience_requireds.append(data['years_of_experience_required'])

    type_counts = Counter(work_types)
    print("\n" + "="*50)
    print("\nwork_types Counts:")
    for type_name, count in type_counts.most_common():
        print(f"{type_name}: {count}")
    print(f"\nTotal count: {sum(type_counts.values())}")

    type_counts = Counter(education_level_requireds)
    print("\n" + "="*50)
    print("\neducation_level_requireds Counts:")
    for type_name, count in type_counts.most_common():
        print(f"{type_name}: {count}")
    print(f"\nTotal count: {sum(type_counts.values())}")

    type_counts = Counter(years_of_experience_requireds)
    print("\n" + "="*50)
    print("\nyears_of_experience_requireds Counts:")
    for type_name, count in type_counts.most_common():
        print(f"{type_name}: {count}")
    print(f"\nTotal count: {sum(type_counts.values())}")

    item_counts = compute_stat_for_multi_items(jobs_data, 'required_languages', include_values=False)
    print("\n" + "="*50)
    print("\nRequired Languages Counts:")
    print(f"\nTotal occurrences: {sum(item_counts.values())}")
    print(f"Total unique languages: {len(item_counts)}")
    for item, count in item_counts.most_common():
        print(f"{item}: {count}")

In [8]:
compute_job_data_stastistics(jobs_data)



work_types Counts:
onsite: 158
remote: 42

Total count: 200


education_level_requireds Counts:
Graduação: 81
Técnico: 60
Tecnólogo: 53
Bacharelado: 6

Total count: 200


years_of_experience_requireds Counts:
1: 61
2: 59
0: 46
3: 34

Total count: 200


Required Languages Counts:

Total occurrences: 254
Total unique languages: 2
Portuguese: 200
English: 54


In [ ]:
def compare_json_content(file1_path: str, file2_path: str) -> dict:
    # Read both JSON files
    with open(file1_path, 'r') as f1:
        data1 = json.load(f1)
    with open(file2_path, 'r') as f2:
        data2 = json.load(f2)
    
    differences = {}
    
    # Compare content for each key present in both files
    common_keys = set(data1.keys()) & set(data2.keys())
    
    for key in sorted(common_keys):
        job1 = json.loads(data1[key])  # Parse the JSON string content
        job2 = json.loads(data2[key])  # Parse the JSON string content
        
        # Compare each field in the job listing
        field_differences = {}
        all_fields = set(job1.keys()) | set(job2.keys())
        
        for field in all_fields:
            value1 = job1.get(field)
            value2 = job2.get(field)
            if value1 != value2:
                field_differences[field] = {
                    'file1': value1,
                    'file2': value2
                }
        
        # Only add to differences if there are actual differences
        if field_differences:
            differences[key] = field_differences
    
    return differences

In [ ]:
def print_differences(differences: dict) -> None:
    if not differences:
        print("No differences found in job contents between the files.")
        return
        
    print(f"\nFound differences in {len(differences)} jobs:")
    for job_key, job_diffs in differences.items():
        print(f"\nJob {job_key}:")
        for field, values in job_diffs.items():
            print(f"  Field '{field}':")
            print(f"    File 1: {values['file1']}")
            print(f"    File 2: {values['file2']}")

In [ ]:
file1_path = DATA_JOBS_DIR / "extracted_jobs_v2.json"
file2_path = DATA_JOBS_DIR / "extracted_jobs_v3.json"
differences = compare_json_content(file1_path, file2_path)
print_differences(differences)

In [ ]:
# Test Job Extraction Agent
print("💼 Testing Job Extraction Agent...")
print("Reading a sample job file...\n")

# Get first job file
job_paths = get_job_paths()
if job_paths:
    print(job_paths[0])
    sample_job = extract_job_info(job_paths[0])
    print(f"Job ID: {job_paths[0].stem}")
    print(sample_job.describe())
    print("\nRaw Pydantic model:")
    print(sample_job.model_dump())
else:
    print("No job files found!")